In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import os
print(os.listdir("../input/fruits-360_dataset/fruits-360"))



['test-multiple_fruits', 'Training', 'readme.md', 'LICENSE', 'papers', 'Test']


In [7]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import TensorDataset,DataLoader

channel_means = (0.485, 0.456, 0.406)
channel_stds = (0.229, 0.224, 0.225)
transformation = transforms.Compose([
        transforms.Resize(size=(150,150)),
        transforms.ToTensor(),
        transforms.Normalize(channel_means,channel_stds )])

batch = 64
path = "../input/fruits-360_dataset/fruits-360/"
train_dataset = torchvision.datasets.ImageFolder(path+"Training", transform=transformation)
train_loader = DataLoader(train_dataset,batch_size=batch, shuffle=True)


In [8]:
test_val = torchvision.datasets.ImageFolder(path + "Test", transform=transformation)
test = int(len(test_val)/2)
val = int(len(test_val) - test)
val_dataset, test_dataset = torch.utils.data.random_split(test_val, [val, test])

val_loader = DataLoader(val_dataset,batch_size=batch, shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=batch, shuffle=True)

print(len(train_dataset),len(val_dataset),len(test_dataset))

53177 8923 8922


In [9]:
import torch.nn as nn
import torch.nn.functional as F
import math
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=50, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(3),
            nn.Conv2d(in_channels=50, out_channels=64, kernel_size=4),
            nn.ReLU(),
            nn.MaxPool2d(4),
            nn.Conv2d(in_channels=64, out_channels=80, kernel_size=2),
            nn.ReLU(),
            nn.MaxPool2d(5))
        
            
        self.layer2 = nn.Sequential(
            nn.Linear(320,128), nn.ReLU(),
            nn.Linear(128,103)
        )
    def forward(self, x):
        y = self.layer2(self.layer1(x).view(x.size(0), -1))
        return y

In [10]:
def fit(model, train_dl,val_dl, lr, epoches,tolerance):
    model.cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    best_loss = 100
    cur_tol = tolerance
    for epoche in range(epoches):
        ep_loss = 0
        for xx,yy in train_dl:
            xx,yy = xx.cuda(), yy.cuda()
            optimizer.zero_grad()
            y_pred = model(xx)
            loss = criterion(y_pred, yy)
            loss.backward()
            ep_loss+=loss.item()
            optimizer.step()
        print("Loss: {}".format(ep_loss/len(train_dl)))
        with torch.no_grad():
            val_loss=0
            for xx,yy in val_dl:
                xx,yy = xx.cuda(), yy.cuda()
                y_pred = model(xx)
                loss = criterion(y_pred, yy)
                val_loss+=loss.item()
            val_loss/=len(val_dl)
            if best_loss>= val_loss:
                best_loss = val_loss
                cur_tol = tolerance
                torch.save(model.state_dict(), "..\bestmodel.mod")
            else:
                cur_tol -= 1
            if cur_tol==0:
                model.load_state_dict(torch.load("..\bestmodel.mod"))
                break
        print("---->Val loss: {}".format(val_loss))
    print("Stop train.")
    model.cpu()

In [11]:
net = NN()
fit(net,train_loader,val_loader,0.005,6,3)

Loss: 0.6618138063994878
---->Val loss: 0.5341633870665516
Loss: 0.14011340851925772
---->Val loss: 0.25222966372966765
Loss: 0.10574461072324702
---->Val loss: 0.4710174143580454
Loss: 0.12019159664006547
---->Val loss: 0.32646183451371535
Loss: 0.08818072749173748
Stop train.


In [12]:
net.load_state_dict(torch.load("..\bestmodel.mod"))
y_true = []
y_pred = []
for xx,yy in test_loader:
    net.cuda()
    xx,yy = xx.cuda(), yy.cuda()
    out = net(xx).argmax(dim=1)
    y_true.extend(yy.tolist())
    y_pred.extend(out.tolist())
net.cpu()

NN(
  (layer1): Sequential(
    (0): Conv2d(3, 50, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(50, 64, kernel_size=(4, 4), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 80, kernel_size=(2, 2), stride=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Linear(in_features=320, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=103, bias=True)
  )
)

In [13]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.45      0.83      0.59        81
           1       0.93      0.84      0.88        79
           2       0.93      0.80      0.86        83
           3       1.00      0.94      0.97        81
           4       1.00      1.00      1.00        84
           5       1.00      1.00      1.00        80
           6       0.86      0.96      0.91        84
           7       0.73      0.73      0.73        90
           8       1.00      0.46      0.63        79
           9       1.00      0.73      0.85        79
          10       0.99      1.00      0.99        83
          11       1.00      0.77      0.87        88
          12       1.00      1.00      1.00       115
          13       1.00      1.00      1.00        97
          14       1.00      1.00      1.00        68
          15       0.89      1.00      0.94        74
          16       1.00      1.00      1.00        83
          17       1.00    